#Assignment #6

Vision Transformer

---


**SUBMISSION INSTRUCTIONS**

First make a copy of this colab file and then solve the assignment and upload your final notebook on github.

Before uploading your downloaded notebook, RENAME the file as rollno_name.ipynb

Submission Deadline : 31/01/2026 Saturday EOD i.e before 11:59 PM

The deadline is strict, Late submissions will incur penalty

Note that you have to upload your solution on the github page of the project Vision Transformer and **under Assignment 6**

And remember to keep title of your pull request to be ViT_name_rollno_assgn6

Github Submission repo -
https://github.com/electricalengineersiitk/Winter-projects-25-26/tree/main/Vision%20transformer/Assignment%206

# Part A - Data and tokens

Q1. Build a tiny toy dataset with pandas
Create a pandas DataFrame with columns text and label.
- Include at least 12 short sentences (3-10 words each).
- The label is 0/1 (e.g., positive vs negative sentiment).
- Shuffle rows and split into train/test (80/20) using a fixed random seed.
Return: df_train, df_test.

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

def make_toy_dataset(seed: int = 42):
    data = {
        "text": [
            "i love this movie", "it was a great film", "highly recommend this",
            "best experience ever", "i enjoyed the show", "the acting was superb",
            "i hate this movie", "it was a terrible film", "do not recommend",
            "worst experience ever", "i disliked the show", "the acting was poor"
        ],
        "label": [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
    }
    df = pd.DataFrame(data)
    df = df.sample(frac=1, random_state=seed).reset_index(drop=True)
    df_train, df_test = train_test_split(df, test_size=0.2, random_state=seed)
    return df_train, df_test

Q2. Clean and tokenize text

Implement a basic cleaner: lowercase, strip, replace multiple spaces with one, and remove punctuation
(.,!?;:).
Tokenize by whitespace.
Add a new column tokens that stores a list of tokens per row.
Return the updated DataFrame.

In [16]:
import re

def clean_text(s: str) -> str:
    s = s.lower().strip()
    s = re.sub(r'[.,!?;:]', '', s)
    s = re.sub(r'\s+', ' ', s)
    return s

def add_tokens_column(df: pd.DataFrame) -> pd.DataFrame:
    df['tokens'] = df['text'].apply(lambda x: clean_text(x).split())
    return df

Q3. Build a vocabulary + token/id mappings

Build token2id and id2token using the training tokens.
Include special tokens: [PAD], [UNK], [BOS], [EOS] at the beginning.
Add tokens that occur at least min_freq times.
Return: token2id (dict), id2token (list).

In [17]:
from collections import Counter

SPECIALS = ['[PAD]', '[UNK]', '[BOS]', '[EOS]']

def build_vocab(list_of_token_lists, min_freq: int = 1):
    counts = Counter([t for tokens in list_of_token_lists for t in tokens])
    vocab = [t for t, f in counts.items() if f >= min_freq]
    id2token = SPECIALS + sorted(vocab)
    token2id = {t: i for i, t in enumerate(id2token)}
    return token2id, id2token

Q4. Convert tokens to ids + pad to a batch

Implement tokens_to_ids for one sequence.
Implement pad_batch that takes a list of id sequences and returns:
- X: int array (B,T) padded with pad_id
- pad_mask: bool array (B,T) where True means 'real token' and False means padding

In [18]:
import numpy as np

def tokens_to_ids(tokens, token2id, unk_token='[UNK]'):
    unk_id = token2id.get(unk_token)
    return [token2id.get(t, unk_id) for t in tokens]

def pad_batch(id_seqs, pad_id: int):
    max_len = max(len(seq) for seq in id_seqs)
    batch_size = len(id_seqs)
    X = np.full((batch_size, max_len), pad_id)
    pad_mask = np.zeros((batch_size, max_len), dtype=bool)

    for i, seq in enumerate(id_seqs):
        X[i, :len(seq)] = seq
        pad_mask[i, :len(seq)] = True
    return X, pad_mask

#Part B - Core Transformer math

Q5. Embedding lookup

Implement an embedding table E of shape (V,D) initialized from a normal distribution (mean 0, std 0.02).
Given token ids X (B,T), return embeddings of shape (B,T,D) using NumPy indexing.


In [19]:
def init_embeddings(vocab_size: int, d_model: int, seed: int = 0):
    np.random.seed(seed)
    return np.random.normal(0, 0.02, (vocab_size, d_model))

def embed(X: np.ndarray, E: np.ndarray):
    return E[X]

Q6. Sinusoidal positional encoding

Implement the classic sinusoidal positional encoding PE of shape (T,D).
Then add it to token embeddings (B,T,D).
Make sure your implementation works for both even and odd D.

In [20]:
def sinusoidal_positional_encoding(T: int, D: int):
    PE = np.zeros((T, D))
    for pos in range(T):
        for i in range(0, D, 2):
            div_term = np.exp(i * -np.log(10000.0) / D)
            PE[pos, i] = np.sin(pos * div_term)
            if i + 1 < D:
                PE[pos, i + 1] = np.cos(pos * div_term)
    return PE

def add_positional_encoding(X_emb: np.ndarray, PE: np.ndarray):
    return X_emb + PE[:X_emb.shape[1], :]

Q7. Scaled dot-product attention with masking

Implement scaled dot-product attention:
Attention(Q,K,V) = softmax((Q @ K^T) / sqrt(dk) + mask) @ V
Inputs: Q,K,V are (B,H,T,Dh). Mask is boolean broadcastable to (B,H,T,T) where False means 'mask out'.
Requirements:
- Use a numerically stable softmax (subtract max).
- Convert boolean mask to large negative values before softmax.
Return: context (B,H,T,Dh) and attention weights (B,H,T,T).

In [21]:
def softmax(x: np.ndarray, axis: int = -1):
    e_x = np.exp(x - np.max(x, axis=axis, keepdims=True))
    return e_x / e_x.sum(axis=axis, keepdims=True)

def scaled_dot_product_attention(Q, K, V, mask=None):
    dk = Q.shape[-1]
    scores = np.matmul(Q, K.transpose(0, 1, 3, 2)) / np.sqrt(dk)
    if mask is not None:
        # mask is (B, 1, 1, T) or (B, 1, T, T)
        scores = np.where(mask, scores, -1e9)
    attn = softmax(scores, axis=-1)
    context = np.matmul(attn, V)
    return context, attn

Q8. Multi-head self-attention (MHA)

Implement multi-head self-attention for input X (B,T,D).
- Project to Q,K,V using weight matrices Wq,Wk,Wv each (D,D).
- Reshape/split into heads -> (B,H,T,Dh) where Dh=D/H.
- Apply scaled dot-product attention with a pad mask (B,T) (broadcast it appropriately).
- Concatenate heads and apply output projection Wo (D,D).
Return: out (B,T,D) and attention weights (B,H,T,T).

In [22]:
def linear(x: np.ndarray, W: np.ndarray, b=None):
    out = np.matmul(x, W)
    return out + b if b is not None else out

def split_heads(x: np.ndarray, n_heads: int):
    B, T, D = x.shape
    Dh = D // n_heads
    return x.reshape(B, T, n_heads, Dh).transpose(0, 2, 1, 3)

def combine_heads(xh: np.ndarray):
    B, H, T, Dh = xh.shape
    return xh.transpose(0, 2, 1, 3).reshape(B, T, H * Dh)

def mha_self_attention(X, Wq, Wk, Wv, Wo, n_heads: int, pad_mask=None):
    Q = split_heads(linear(X, Wq), n_heads)
    K = split_heads(linear(X, Wk), n_heads)
    V = split_heads(linear(X, Wv), n_heads)

    # Broadcast pad_mask (B, T) -> (B, 1, 1, T)
    m = pad_mask[:, None, None, :] if pad_mask is not None else None
    context, attn = scaled_dot_product_attention(Q, K, V, mask=m)

    out = linear(combine_heads(context), Wo)
    return out, attn

Q9. LayerNorm + residual connection

Implement LayerNorm for X (B,T,D) using learnable gamma and beta of shape (D,).
Then implement residual_add_and_norm(Y, X, gamma, beta) that returns LayerNorm(X + Y).

In [23]:
def layer_norm(X: np.ndarray, gamma: np.ndarray, beta: np.ndarray, eps: float = 1e-5):
    mean = X.mean(axis=-1, keepdims=True)
    var = X.var(axis=-1, keepdims=True)
    return gamma * (X - mean) / np.sqrt(var + eps) + beta

def residual_add_and_norm(Y, X, gamma, beta):
    return layer_norm(X + Y, gamma, beta)

Q10. Position-wise FeedForward network

Implement FFN: FFN(X) = relu(X @ W1 + b1) @ W2 + b2
Shapes: X (B,T,D), W1 (D,Dff), b1 (Dff,), W2 (Dff,D), b2 (D,)
Return: (B,T,D).

In [24]:
def relu(x: np.ndarray):
    return np.maximum(0, x)

def feed_forward(X, W1, b1, W2, b2):
    return linear(relu(linear(X, W1, b1)), W2, b2)

# Part C - Putting it together

Q11. One Transformer encoder block (forward)

Implement a single encoder block forward pass:
1) MHA = MultiHeadSelfAttention(X) with pad_mask
2) X1 = LayerNorm(X + MHA)
3) FFN = FeedForward(X1)
4) X2 = LayerNorm(X1 + FFN)
Return X2.
You may pass all parameters explicitly (weights, gamma/beta).

In [25]:
def encoder_block_forward(X, params, n_heads: int, pad_mask=None):
    # 1 & 2: MHA + Norm
    mha_out, _ = mha_self_attention(X, params['Wq'], params['Wk'], params['Wv'],
                                   params['Wo'], n_heads, pad_mask)
    X1 = residual_add_and_norm(mha_out, X, params['gamma1'], params['beta1'])

    # 3 & 4: FFN + Norm
    ffn_out = feed_forward(X1, params['W1'], params['b1'], params['W2'], params['b2'])
    X2 = residual_add_and_norm(ffn_out, X1, params['gamma2'], params['beta2'])
    return X2

Q12. Sequence classification head + end-to-end demo

Create an end-to-end forward pass for a tiny classifier:
- Input ids -> embeddings + positional enc
- One encoder block
- Pooling: take the [BOS] position (t=0) as the sequence representation
- Linear head: logits = h0 @ Wcls + bcls with Wcls (D,2), bcls (2,)
- Softmax to probabilities
Write predict_proba that takes a batch of texts and returns probs (B,2).
Include simple sanity checks: shapes, probabilities sum to 1, and masking doesn't crash for different
lengths.


In [26]:
def predict_proba(texts, token2id, E, PE, params, Wcls, bcls, n_heads: int):
    # Prepare batch
    clean_texts = [clean_text(t).split() for t in texts]
    # Adding [BOS] and [EOS]
    seqs = [[token2id['[BOS]']] + tokens_to_ids(ts, token2id) + [token2id['[EOS]']] for ts in clean_texts]
    X_ids, pad_mask = pad_batch(seqs, token2id['[PAD]'])

    # Forward Pass
    X_emb = embed(X_ids, E)
    X_pe = add_positional_encoding(X_emb, PE)
    X_enc = encoder_block_forward(X_pe, params, n_heads, pad_mask)

    # Pooling
    h0 = X_enc[:, 0, :]
    logits = linear(h0, Wcls, bcls)
    probs = softmax(logits, axis=-1)

    return probs

In [28]:
df_tr, df_ts = make_toy_dataset()
t2id, _ = build_vocab(add_tokens_column(df_tr)['tokens'].tolist())

D, H, V = 16, 2, len(t2id)
E, PE = init_embeddings(V, D), sinusoidal_positional_encoding(20, D)

params = {
    'Wq': np.random.randn(D, D) * 0.02,
    'Wk': np.random.randn(D, D) * 0.02,
    'Wv': np.random.randn(D, D) * 0.02,
    'Wo': np.random.randn(D, D) * 0.02,
    'gamma1': np.ones(D), 'beta1': np.zeros(D),
    'gamma2': np.ones(D), 'beta2': np.zeros(D),
    'W1': np.random.randn(D, 4*D) * 0.02, 'b1': np.zeros(4*D),
    'W2': np.random.randn(4*D, D) * 0.02, 'b2': np.zeros(D)
}

Wcls, bcls = np.random.randn(D, 2) * 0.02, np.zeros(2)

# Run prediction
probs = predict_proba(df_ts['text'].tolist()[:3], t2id, E, PE, params, Wcls, bcls, H)

print("Probs:\n", probs)
print("Sum check:", probs.sum(1))

Probs:
 [[0.56128372 0.43871628]
 [0.56129093 0.43870907]
 [0.56128726 0.43871274]]
Sum check: [1. 1. 1.]
